In [91]:
import csv
import xml.etree.ElementTree as ET

def scrapeRawData(fileName):
    tree = ET.parse(fileName)
    root = tree.getroot()

#     with open('output.csv', 'w') as f:
#         writer = csv.writer(f)
#         writer.writerow(('lat', 'lon'))
#         root = lxml.etree.fromstring(x)
#         for trkpt in root.iter('trkpt'):
#             row = trkpt.get('lat'), trkpt.get('lon')
#             writer.writerow(row)
def getBearings(fileName):
    tree = ET.parse(fileName)
    records = tree.getroot()
    entries = []
    print("root.tag: ", records.tag)
    print("root.attrib: ", records.attrib)
    for result in records:
        # Traverse headers
        header = result.find('header')
        infoMaster = header.find('controlInfo')

        artInfo = infoMaster.find('artinfo')
        # Get title     
        tig = artInfo.find('tig')
        title = tig.find('atl').text
        
        # Get author
        aug = artInfo.find('aug')
        aus = aug.findall('au')
        authors = []
        for au in aus:
            authors.append(au.text)
        
        
        # Get journal names
        jInfo = infoMaster.find('jinfo')
        journalName = jInfo.find('jtl').text
    
        # get language
        language = infoMaster.find('language').text

        
        # Get publication info
        pubInfo = infoMaster.find('pubinfo')
        dateTime = pubInfo.find('dt')
        att = dateTime.attrib
        year = att.get('year')
        month = att.get('month')
        day = att.get('day')
        date = {"month": month,
                "day": day,
                "year": year}
        
        # Pack results into entries
        entry = {"title": title,
                "authors": authors,
                "datePublished": date,
                "authors": authors,
                "language": language,
                "journalName": journalName}
        
        entries.append(entry)
    return entries

In [92]:
scrapeRawData("rawData.xml");
getBearings("rawData.xml")

root.tag:  records
root.attrib:  {}


[{'title': 'Editorial corner - a personal view The coronavirus and plastics.',
  'authors': ['Czigány, T.', 'Ronkay, F.'],
  'datePublished': {'month': '06', 'day': '01', 'year': '2020'},
  'language': 'English',
  'journalName': 'Express Polymer Letters'},
 {'title': 'Pediatricians are ready for COVID‐19 and other emergencies.',
  'authors': ['Knopf, Alison'],
  'datePublished': {'month': '05', 'day': '01', 'year': '2020'},
  'language': 'English',
  'journalName': 'Brown University Child & Adolescent Behavior Letter'},
 {'title': 'International Guidelines on Radiation Therapy for Breast Cancer During the COVID-19 Pandemic.',
  'authors': ['Coles, C.E.',
   'Aristei, C.',
   'Bliss, J.',
   'Boersma, L.',
   'Brunt, A.M.',
   'Chatterjee, S.',
   'Hanna, G.',
   'Jagsi, R.',
   'Kaidar Person, O.',
   'Kirby, A.',
   'Mjaaland, I.',
   'Meattini, I.',
   'Luis, A.M.',
   'Marta, G.N.',
   'Offersen, B.',
   'Poortmans, P.',
   'Rivera, S.'],
  'datePublished': {'month': '05', 'day': '